In [1]:
#import libraries
from keras import models, layers
import numpy as np
from keras.utils import to_categorical
import pandas as pd
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Dense, Lambda
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
#load data
data = pd.read_csv('train.csv')

In [3]:
# Modifying the values of each pixel such that they range from 0 to 1 
# will improve the rate at which our model learns.
df_X, df_y= data.iloc[:,1:]/255, data.iloc[:,0] 
# (n_images, x_shape, y_shape, channels)
# channels = (1 for grayscale images) or (1 for a set of RGB-images) as input
# n_images = -1 is the same as specifying the total number of images in the training set.
# reshaping 784 to 28,28,1
df_X = df_X.values.reshape(data.shape[0],28,28,1)
# One hot encoding is a process by which categorical variables are converted 
# into a form that could be provided to ML algorithms to do a better job in prediction
df_y = to_categorical(df_y)

In [4]:
#splits train/test set 
train_X, val_X, train_y, val_y = train_test_split(df_X, df_y, test_size =0.2,random_state=21)
train_X.shape, train_y.shape, val_X.shape, val_y.shape

((33600, 28, 28, 1), (33600, 10), (8400, 28, 28, 1), (8400, 10))

In [5]:
#normalize the batch manually
mean_x = train_X.mean().astype(np.float32)
std_x = train_X.std().astype(np.float32)

In [6]:
#defining the model
# 32,64,128,128 neurons (feature maps) 
# 3x3 feature detector 
# 2x2 pooling layers 
def cnn():
    model = models.Sequential()
    
    # standard score = (x - mean) / standard deviation 
    model.add(Lambda(lambda x:(x-mean_x)/std_x,input_shape=(28,28,1)))
    
    model.add(Convolution2D(32,(3,3), activation = 'relu'))
    model.add(BatchNormalization(axis=1))   
    
    model.add(Convolution2D(64,(3,3), activation = 'relu'))
    model.add(MaxPooling2D())
    
    model.add(Convolution2D(128,(3,3), activation = 'relu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(128,(3,3), activation = 'relu'))
    model.add(MaxPooling2D())
    
    # The input to our artificial neural network must be in one dimension 
    # therefore we flatten it beforehand
    model.add(Flatten())
    
    # Dense output = activation(dot(input, weight) + bias)
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(10, activation='softmax'))          
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model  

cnn_model = cnn()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#fitting the model
# The batch size is a number of samples processed before the model is updated.
# The number of epochs is the number of complete passes through the training dataset.
cnn_model.fit(train_X, train_y, epochs=10,batch_size=1000,validation_data=(val_X,val_y))

Instructions for updating:
Use tf.cast instead.
Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 72s 2ms/step - loss: 1.0716 - acc: 0.6669 - val_loss: 0.1503 - val_acc: 0.9571
Epoch 2/10
33600/33600 [==============================] - 69s 2ms/step - loss: 0.1843 - acc: 0.9445 - val_loss: 0.0742 - val_acc: 0.9781
Epoch 3/10
33600/33600 [==============================] - 69s 2ms/step - loss: 0.1044 - acc: 0.9682 - val_loss: 0.0555 - val_acc: 0.9843
Epoch 4/10
33600/33600 [==============================] - 70s 2ms/step - loss: 0.0766 - acc: 0.9781 - val_loss: 0.0486 - val_acc: 0.9865
Epoch 5/10
33600/33600 [==============================] - 70s 2ms/step - loss: 0.0588 - acc: 0.9818 - val_loss: 0.0424 - val_acc: 0.9875
Epoch 6/10
33600/33600 [==============================] - 70s 2ms/step - loss: 0.0493 - acc: 0.9848 - val_loss: 0.0428 - val_acc: 0.9887
Epoch 7/10
33600/33600 [==============================] - 69s 2ms/step - loss: 0.

In [8]:
#prediction
cnn_predictions = cnn_model.predict(val_X) 
cnn_predictions = np.argmax(cnn_predictions, axis=1)
print(cnn_predictions)

[1 4 2 ... 0 1 2]


In [9]:
#OutPut
out=pd.DataFrame({"ImageId": list(range(1,len(cnn_predictions)+1)),
                         "Label": cnn_predictions})
out.to_csv("cnn_out.csv", index=False, header=True)
out.shape

(8400, 2)